# Задание 1.
**Обязательная часть**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

**Дополнительная часть (необязательная)**

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
import requests
from bs4 import BeautifulSoup

KEYWORDS = ['python', 'it']

res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
# Фрмируем список ссылок на статьи

all_links = []
for tag_a in soup.select('.tm-article-snippet__readmore'): # ссылка на статью по кнопке "Читать далее", класс в разметке 'tm-article-snippet__readmore'
  all_links.append('https://habr.com' + tag_a.get('href'))

In [ ]:
import time
import pandas as pd

news = pd.DataFrame()

# перебираем в цикле каждую ссылку на статью. фактически просто заходим по каждой ссылке
for link in all_links:
  time.sleep(0.3)
  post = requests.get(link)
  soup_post = BeautifulSoup(post.text, 'html.parser')
  
  # получаем текст статьи
  article = soup_post.find('div', class_='article-formatted-body').text
  
  # проверка на наличие в тексте искомых слов
  if any([word in article.lower() for word in KEYWORDS]):
     date = soup_post.find('time')['title'] # дата хранится в теге <time>, аттрибут 'title'
     title = soup_post.find('h1').text # заголовок в теге 'h1'

     # формируем датафрейм
     row = {'date': date, 'title': title, 'link': link, 'text': article}
     news = pd.concat([news, pd.DataFrame([row])])
  
news

,date,title,link,text
0,"2021-09-19, 13:20",Тест примечательной акустики: два экспертных м...,https://habr.com/ru/company/audiomania/blog/57...,На прошлой неделе мы представили обзоры заметн...
0,"2021-09-19, 13:00",В чём разница между Debian и Ubuntu? Что лучше...,https://habr.com/ru/company/ruvds/blog/576380/,"Поговорим о Debian и Ubuntu. И там, и там можн..."
0,"2021-09-19, 12:38",Как я проходила очередное собеседование и не п...,https://habr.com/ru/post/578784/,"Рисунок: @shato_illustПривет! Меня зовут Саша,..."
0,"2021-09-19, 12:32",Высокопроизводительная вставка с помощью SqlBu...,https://habr.com/ru/post/578780/,ВступлениеЭто перевод статьи Yuria Magurdumova...
0,"2021-09-19, 12:29",Мониторинг оборудования Eltex по протоколу SNMPv3,https://habr.com/ru/post/578766/,В этой статье речь пойдет про мониторинг обору...
0,"2021-09-19, 11:24",Многоконсольный вывод для Python,https://habr.com/ru/post/578772/,Мотив создания ...Существует много проекты в к...
0,"2021-09-19, 04:45",Кросс компиляция пакета для роутера на примере...,https://habr.com/ru/post/578758/,Добрый день друзья. Как-то мне захотелось уста...
0,"2021-09-19, 01:41",Про многопоточность 2. GCD,https://habr.com/ru/post/578752/,Привет! Вот и новая часть серии статей про мно...
0,"2021-09-19, 01:02",Точка пересечения двух отрезков,https://habr.com/ru/post/578746/,ВведениеУже не раз на habr затрагивалась данна...
0,"2021-09-18, 23:18",ВМС США убирает в дальний ящик создание Railgun,https://habr.com/ru/post/576624/,"\n\r\nЛюбой, кто играл в Quake, знает о рельсо..."


# Задание 2.
**Обязательная часть**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

In [ ]:
import requests
import json

EMAIL = ['xxx@x.ru', 'yyy@y.com']

payload = {'emailAddresses': EMAIL}

headers = {'Vaar-Header-App-Build-Version': '1.0.0',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'}

result = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', data=json.dumps(payload), headers=headers)

In [ ]:
import pandas as pd

dict_res = json.loads(result.text)

leaks = pd.DataFrame()

for breach in dict_res['breaches'].values():
  row = {'date':breach['publishDate'], 'sourse_leak': breach['site'], 'description': breach['description']}
  leaks = pd.concat([leaks, pd.DataFrame([row])])

leaks

,date,sourse_leak,description
0,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
0,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
0,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
0,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."


# Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology’

TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [ ]:
import requests

GROUP = 'netology'

with open('token_vk.txt', 'r') as file_token:
  TOKEN = file_token.read().strip()

params = {'access_token': TOKEN,
          'v': '5.131',
          'domain': GROUP,
          'count': '50'
          }

posts = requests.get('https://api.vk.com/method/wall.get', params=params)

In [ ]:
posts_dict = posts.json()

posts_group = pd.DataFrame()

for post in posts_dict['response']['items']:
    date = pd.to_datetime(post['date'], unit='s') # время без учета часового пояса
    row = {'date':date, 'post_text': post['text']}
    posts_group = pd.concat([posts_group, pd.DataFrame([row])])

posts_group

,date,post_text
0,2021-09-15 14:15:00,"Поговорили с коллегами и выяснили, из чего на ..."
0,2021-09-19 07:45:00,Токсичные люди — мастера отравлять жизнь окруж...
0,2021-09-18 08:41:00,"Подготовиться к презентации или собеседованию,..."
0,2021-09-17 15:00:02,Совет: ошибки на старте карьеры и не только — ...
0,2021-09-17 07:23:00,"Учиться, изучать рынок и продвигать — на занят..."
0,2021-09-16 15:57:00,Валентин Перевозчиков решил сменить профессию ...
0,2021-09-16 08:59:18,Если в офлайн-обучении нетворкинг проходит сам...
0,2021-09-15 07:37:00,👩‍💻 Запустили бесплатный курс «Основы вёрстки ...
0,2021-09-14 15:13:00,"Спойлер: секрет, в принципе, всего один. Когда..."
0,2021-09-14 09:01:00,"Когда сервисы уже не предлагают хорошее кино, ..."
